In [1]:
# Split by HTML header

'MarkdownHeaderTextSplitter'，'HTMLHeaderTextSplitter'是一个“结构感知”分块器，它在元素级别拆分文本，并为每个标题添加元数据“相关”到任何给定块。它可以逐个元素返回块，也可以将具有相同元数据的元素组合在一起，其目标是 （a） 在语义上对相关文本进行分组（或多或少），以及 （b） 保留在文档结构中编码的上下文丰富的信息。它可以与其他文本拆分器一起使用，作为分块管道的一部分。

## 使用示例

1）使用HTML字符串：

In [1]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text(html_string)
html_header_splits

[Document(page_content='Foo'),
 Document(page_content='Some intro text about Foo.  \nBar main section Bar subsection 1 Bar subsection 2', metadata={'Header 1': 'Foo'}),
 Document(page_content='Some intro text about Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section'}),
 Document(page_content='Some text about the first subtopic of Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 1'}),
 Document(page_content='Some text about the second subtopic of Bar.', metadata={'Header 1': 'Foo', 'Header 2': 'Bar main section', 'Header 3': 'Bar subsection 2'}),
 Document(page_content='Baz', metadata={'Header 1': 'Foo'}),
 Document(page_content='Some text about Baz', metadata={'Header 1': 'Foo', 'Header 2': 'Baz'}),
 Document(page_content='Some concluding text about Foo', metadata={'Header 1': 'Foo'})]

2） 通过管道传输到另一个拆分器，从 Web URL 加载 html：

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# url = "https://plato.stanford.edu/entries/goedel/"
url = "https://zh.wikipedia.org/wiki/%E5%8A%89%E5%BE%B7%E8%8F%AF"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
    ("h4", "Header 4"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# for local file use html_splitter.split_text_from_file(<path_to_file>)
html_header_splits = html_splitter.split_text_from_url(url)

chunk_size = 500
chunk_overlap = 30
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

# Split
splits = text_splitter.split_documents(html_header_splits)
splits[80:85]

[Document(page_content='主条目：劉德華演唱會列表', metadata={'Header 1': '劉德華', 'Header 2': '歌唱事业[编辑]', 'Header 3': '演唱会[编辑]'}),
 Document(page_content='1991年，刘德华凭借其在电影方面的影响力，在韩国汉城（今首尔）首爾奧林匹克體操競技場举办了一场演唱会，这是他举办的首场个人演唱会。1992年赴美国与加拿大举办的八场个唱为其首次巡回演出[189]。1993年1月，首次於紅磡體育館舉辦了20场的在港個人演唱會[142]。之后1994年与1996年又各举办了20场，1999年、2001年与2004年的夏季各举办15场，2007年、2010年和2018年年底各举办了16场、20场和13场（2018年原定20场，因喉咙发炎被迫取消了7场）的跨年演出[188][198]，累计起来目前他已在香港红馆开唱154场。1993年刘德华首次到中国大陆展开了23场的室内巡回演唱会[189]，为其首次中国巡演活动。1997年与1998年在内地累计举办20场[189]；2000年的中国巡演在济南、沈阳等10个城市展开[189]。2001年与2002年的“夏日Fiesta”巡演合计在武汉、大连、南昌、西安等10个城市举行[189]。2004年至2005年的“Vision', metadata={'Header 1': '劉德華', 'Header 2': '歌唱事业[编辑]', 'Header 3': '演唱会[编辑]'}),
 Document(page_content='Tour”中国巡回在上海、苏州、西安、无锡等19个城市上演[189]。2007年的“Wonderful World”中国巡回从呼和浩特开始到成都结束共在15个城市举办[193]，其中上海站一连举办了两场；2009年继续进行了广州、合肥等9个城市的“Wonderful World”中国巡回个唱[192]。2011年在北京、郑州和天津等11站举办“Unforgettable”中国巡回演出[199]。2013年开启的“ALways”中国巡回个唱选择在上海、南京、广州、北京和大连这五座城市的体育馆举行[232]，以达到同香港红馆四面台一样的室内效果。至今刘德华已在中国大陆各城市举办了132场个人演唱会，

## 局限性

从一个 HTML 文档到另一个 HTML 文档可能会有相当多的结构变化，虽然 HTMLHeaderTextSplitter 会尝试将所有“相关”标头附加到任何给定的块，但它有时会遗漏某些标头。例如，该算法假定了一个信息层次结构，其中标头始终位于关联文本“上方”的节点上，即先前的兄弟姐妹、祖先及其组合。在下面的新闻文章中（截至撰写本文档时），文档的结构使得顶级标题的文本虽然标记为“h1”，但与我们期望它位于“上方”的文本元素不同，因此我们可以观察到“h1”元素及其关联文本不会显示在块元数据中（但是， 在适用的情况下，我们确实可以看到“H2”及其相关文本）：

In [11]:
url = "https://www.cnn.com/2023/09/25/weather/el-nino-winter-us-climate/index.html"

headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
html_header_splits = html_splitter.split_text_from_url(url)
print(html_header_splits[1].page_content[:500])

No two El Niño winters are the same, but many have temperature and precipitation trends in common.  
Average conditions during an El Niño winter across the continental US.  
One of the major reasons is the position of the jet stream, which often shifts south during an El Niño winter. This shift typically brings wetter and cooler weather to the South while the North becomes drier and warmer, according to NOAA.  
Because the jet stream is essentially a river of air that storms flow through, they c
